# load samples from nerdpool-api and train NER

In [1]:
import json
from nerdpool_client import NerdPoolClient

In [2]:
client = NerdPoolClient()

In [3]:
client.dump_to_jsonl("https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&ner_ent_type__contains=&ner_source__title=MRP")

https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&ner_ent_type__contains=&ner_source__title=MRP
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=100
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=200
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=300
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=400
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=500
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=600
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&

https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6200
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6300
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6400
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6500
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6600
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6700
https://nerdpool-api.acdh-dev.oeaw.ac.at/api/ner-sample/?format=json&limit=100&ner_ent_type__contains=&ner_source__title=MRP&offset=6800
https://nerdpool-api.acdh-dev.oeaw.ac.at/

'out.jsonl'

In [4]:
import warnings
import spacy
from spacy.tokens import DocBin

In [5]:
nlp = spacy.blank('de')
db_train = DocBin()
db_eval = DocBin()
fails = 0
train_eval = 0
train_sample_count = 0
eval_sample_count = 0
with open('out.jsonl') as f:
    for line in f:
        if train_eval == 5:
            train_eval = 0
        try:
            data = json.loads(line.strip())
        except:
            continue
        text = data['text']
        entities = data['entities']
        doc = nlp.make_doc(text)
        ents = []
        train_eval += 1
        for start, end, label in entities:
            span = doc.char_span(start, end, label=label)
            if span is None:
                # msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                # warnings.warn(msg)
                fails += 1
            else:
                ents.append(span)
        try:
            doc.ents = ents
            if train_eval != 4:
                db_train.add(doc)
                train_sample_count += 1
            else:
                db_eval.add(doc)
                eval_sample_count += 1
        except ValueError:
            continue
    db_train.to_disk('train.spacy')
    db_eval.to_disk('dev.spacy')
print(
    f"fails: {fails}; train: {train_sample_count}; eval: {eval_sample_count}"
)
        

fails: 2; train: 8255; eval: 2064


In [6]:
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-07-14 09:46:43,976] [INFO] Set up nlp object from config
[2021-07-14 09:46:43,984] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-07-14 09:46:43,987] [INFO] Created vocabulary
[2021-07-14 09:46:43,987] [INFO] Finished initializing nlp object
[2021-07-14 09:46:52,214] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     36.28    0.00    0.00    0.00    0.00
  0     200         62.31   1706.05   45.33   71.59   33.16    0.45
  0     400         70.50    946.91   67.80   79.29   59.22    0.68
  0     600        179.24    952.93   74.31   83.40   67.01    0.74
  0     800 